### Assignment 8: Sentence Encoder

6610402230  Sirisuk Tharntham

Step 1 : download pre-trained USE

In [30]:
import os, random, numpy as np
import tensorflow as tf

seed_value = 7777
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

In [31]:
# !pip install tensorflow_text

import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import tensorflow as tf

embed =hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [32]:
sentences = ['ข้าวขาหมูร้านนี้อร่อยมาก']
embed(sentences)

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-1.56073486e-02,  4.83928397e-02,  6.76209480e-02,
         3.26975957e-02, -2.33572293e-02, -3.89391854e-02,
        -5.13572618e-02,  2.53386963e-02,  3.35361399e-02,
         5.70191368e-02, -4.18839529e-02,  6.28810152e-02,
         1.22976843e-02,  9.15155187e-02, -6.71719387e-02,
        -9.03077126e-02,  2.52133682e-02, -4.67808992e-02,
         2.70937681e-02, -5.61775779e-03,  5.44996671e-02,
         4.17851465e-04, -8.46899301e-02,  2.13087332e-02,
         2.20608115e-02,  8.41895060e-04, -2.65123621e-02,
        -6.56712241e-03,  5.01014665e-02,  1.99158043e-02,
         7.11926073e-02,  1.41542414e-02, -1.62930763e-03,
        -5.25229611e-02, -8.76291376e-03, -6.64196610e-02,
         3.42438966e-02,  2.08897162e-02, -2.18446255e-02,
        -5.67424931e-02,  6.26238901e-03, -1.49302799e-02,
        -4.98546287e-02, -1.86504945e-02,  2.17800699e-02,
         1.44530162e-02,  6.02621250e-02,  1.20022744e-02,
      

Step 2 : read file / data

In [33]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1K0VgdwYagNidu5k_y5pnSrGrGIOBPsyS' -O Fastfood_Opinion.csv

--2024-08-28 19:00:28--  https://docs.google.com/uc?export=download&id=1K0VgdwYagNidu5k_y5pnSrGrGIOBPsyS
Resolving docs.google.com (docs.google.com)... 2404:6800:4001:80b::200e, 172.217.26.78
Connecting to docs.google.com (docs.google.com)|2404:6800:4001:80b::200e|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1K0VgdwYagNidu5k_y5pnSrGrGIOBPsyS&export=download [following]
--2024-08-28 19:00:28--  https://drive.usercontent.google.com/download?id=1K0VgdwYagNidu5k_y5pnSrGrGIOBPsyS&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 2404:6800:4001:80a::2001, 216.58.200.1
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|2404:6800:4001:80a::2001|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348315 (340K) [application/octet-stream]
Saving to: ‘Fastfood_Opinion.csv’

Fastfood_Opinion.cs 100%[===================>] 340.15K  

In [34]:
# Step 1: Load and preprocess the data
import pandas as pd
data = pd.read_csv("Fastfood_Opinion.csv",  encoding='utf8')
data.info()

X = data['message'].astype(str)
y = data['class']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   class    642 non-null    int64 
 1   message  642 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.2+ KB


In [83]:
y

0      0
1      0
2      0
3      0
4      0
      ..
637    1
638    1
639    1
640    1
641    1
Name: class, Length: 642, dtype: int64

Step 3 : Encode / Embed

In [35]:
emb_X = embed(X).numpy()
df_emb_X = pd.DataFrame(emb_X, index = X)

In [36]:
category = ['พนักงานบริการดี ใส่ใจลูกค้า', 'อาหารอร่อย ร้อน สดใหม่', 'ราคาคุ้มค่า', 'โปรโมชันดี', 'พนักงานบริการแย่ ไม่สนใจลูกค้า พูดจาไม่ดี', 'อาหารไม่อร่อย ไม่ร้อน ไม่สด เก่า', 'ราคาแพง ไม่คุ้มค่ากับเงินที่จ่าย']

df_emb_category = pd.DataFrame(embed(category).numpy(), index = category)

In [37]:
argmax_category = df_emb_X.dot(df_emb_category.T).idxmax('columns').values
df_result = pd.DataFrame({'original_text' : df_emb_X.index, 'category' : argmax_category})

In [38]:
df_result.sample(10)

,original_text,category
97,กินประจำเลยค่ะเนื้ออร่อย ฟุดๆ,อาหารไม่อร่อย ไม่ร้อน ไม่สด เก่า
590,เป็นพิซซ่าที่อร่อยมากที่สุดกินมาหลายสิบปีละ,อาหารไม่อร่อย ไม่ร้อน ไม่สด เก่า
138,พนักงานน่ารักมากเลยค่ะ เราซื้อเสร็จเดินออกจากร...,พนักงานบริการดี ใส่ใจลูกค้า
108,ชอบค่ะ กลางคืนโอเคเลยนะ คนไม่เยอะ พนักงานบริการดี,พนักงานบริการแย่ ไม่สนใจลูกค้า พูดจาไม่ดี
587,พิซซ่าอร่อยทุกชิ้น แป้งบางกำลังดี น้ำจิ้มข้าวม...,พนักงานบริการดี ใส่ใจลูกค้า
126,น่าทานอร่อยดีครับ,อาหารอร่อย ร้อน สดใหม่
174,อร่อยมากก,อาหารอร่อย ร้อน สดใหม่
13,คืนวันที่ 26 มค. 60 ไปใช้บริการ Burger King สา...,พนักงานบริการแย่ ไม่สนใจลูกค้า พูดจาไม่ดี
42,มีเส้นอย่างอื่นที่ไม่ใช่เส้นผมอยู่ในห่อfish n ...,อาหารไม่อร่อย ไม่ร้อน ไม่สด เก่า
347,เคยเจอเหมือนกันพนักงานเหวี่ยงใส่ ที่สาขาเชลทรั...,พนักงานบริการดี ใส่ใจลูกค้า


Split Data

In [39]:
df_emb_category.shape

(7, 512)

In [57]:
embed_Y = df_result["category"].astype(str)

In [58]:
embed_Y = embed(embed_Y).numpy()

In [60]:
emb_X.shape,embed_Y.shape

((642, 512), (642, 512))

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, y, test_size=0.30, random_state=seed_value)

In [85]:
X_train.shape[1]

512

In [86]:
y_train

607    1
201    0
362    1
328    0
269    0
      ..
395    1
437    1
344    0
16     0
198    0
Name: class, Length: 449, dtype: int64

In [87]:
from keras import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional
from sklearn.metrics import confusion_matrix

model1 = Sequential()
model1.add(Dense(32, input_shape=(X_train.shape[1],),activation="relu"))
model1.add(Dense(16, activation="relu"))
model1.add(Dense(1, activation="sigmoid"))

model1.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

model1.summary()

/Users/sirisuk/anaconda3/envs/nlp_py3.9/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 32)             │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,961 (66.25 KB)

 Trainable params: 16,961 (66.25 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:
model1.fit(X_train,y_train,epochs=10,batch_size=32)

Epoch 1/10


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - accuracy: 0.6204 - loss: 0.6840
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - accuracy: 0.8646 - loss: 0.6270
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.8978 - loss: 0.5482
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - accuracy: 0.9196 - loss: 0.4573
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.9249 - loss: 0.3695
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - accuracy: 0.9387 - loss: 0.2962
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.9426 - loss: 0.2435
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step - accuracy: 0.9506 - loss: 0.2070
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 535us/step - accuracy: 0.9558 - loss: 0.1806
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step - accuracy: 0.9598 - loss: 0.1606


In [89]:
# Evaluate the model
loss, accuracy = model1.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.9081 - loss: 0.2182
Test Accuracy: 0.9067
